[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/itmorn/AI.handbook/blob/main/DL/torch/nn/Activations/MultiheadAttention.ipynb)

# MultiheadAttention （多头自注意力层）
在自注意力机制中，通过将输入序列中每个位置的向量映射到一个query向量、一个key向量和一个value向量上，然后计算query和key之间的相似度得到权重，最后使用这些权重加权得到value向量的线性组合作为输出向量。

而MultiheadAttention将使用多个头(head)，每个头独立地执行上述自注意力机制，并将所有头的输出结果进行拼接。这种方式可以捕获多个不同的语义信息，使得模型更加全面地理解输入序列。

在实践中，MultiheadAttention常常和Transformer结构一起使用，用于处理序列数据。

**定义**：  
torch.nn.MultiheadAttention(embed_dim, num_heads, dropout=0.0, bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None, batch_first=False, device=None, dtype=None)

**参数**:  
- embed_dim – Total dimension of the model. 输入序列中每个时间步的向量维度。

- num_heads – Number of parallel attention heads. Note that embed_dim will be split across num_heads (i.e. each head will have dimension embed_dim // num_heads).  头的数量。注意embed_dim必须可以整除num_heads。

- dropout – Dropout probability on attn_output_weights. Default: 0.0 (no dropout). q和k求完attention后，执行Dropout时的丢弃率

- bias – If specified, adds bias to input / output projection layers. Default: True. 由a产生qkv时有一个线性层，多头汇聚之后，还有一个线性层，这两个层是否加偏置项

- add_bias_kv – If specified, adds bias to the key and value sequences at dim=0. Default: False.  也是一种加bias的方式，只不过比较特殊。它是当做一个时间步加进去的（没有q只有kv），详情见下文。

- add_zero_attn – If specified, adds a new batch of zeros to the key and value sequences at dim=1. Default: False.   能让模型知道 哪些位置是对任务不重要的，详情见下文。

- kdim – Total number of features for keys. Default: None (uses kdim=embed_dim).  一般使用默认值

- vdim – Total number of features for values. Default: None (uses vdim=embed_dim).  一般使用默认值

- batch_first – If True, then the input and output tensors are provided as (batch, seq, feature). Default: False (seq, batch, feature).   输入维度的顺序


# 图解MultiheadAttention 
<p align="center">
<img src="./imgs/MultiheadAttention.svg"
    width="2000" /></p>


In [20]:
import torch
import torch.nn as nn

torch.manual_seed(666)
src = torch.tensor([[[0.2180, 0.6484, 0.0964, 0.0614]],
                    [[0.3732, 0.9106, 0.7293, 0.6012]],
                    [[0.8778, 0.5302, 0.5404, 0.2252]]])  # L=3, N=1, E=4 
# where L is the target sequence length, N is the batch size, and E is the embedding dimension(embed_dim).

multihead_attn = nn.MultiheadAttention(embed_dim=4, num_heads=2,dropout=0.1)
attn_output, attn_output_weights = multihead_attn(src, src, src)
attn_output # 可以看到和上图的结果是一致的

tensor([[[-0.1416, -0.0677, -0.2261, -0.1194]],

        [[-0.1405, -0.0675, -0.2281, -0.1216]],

        [[-0.1408, -0.0670, -0.2294, -0.1232]]], grad_fn=<ViewBackward0>)

# 图解add_bias_kv  
add_bias_kv是指向自注意力机制（self-attention）中的键值对输入添加一个偏置向量。在自注意力机制中，键值对是用于计算注意力权重的关键信息。add_bias_kv技术的目的是在自注意力机制中为键值对添加一个偏置项，以提高模型的准确性和泛化能力。

在Transformer等深度学习模型中，add_bias_kv技术通常是在多头自注意力机制（multi-head self-attention）中使用的。在这种情况下，每个注意力头都具有自己的一组键值对，这些键值对可以并行计算，从而提高计算效率。在使用add_bias_kv技术时，可以为每个注意力头的键和值都添加一个不同的偏置向量，以提高模型的表达能力。

具体来说，add_bias_kv技术会向输入的键值对矩阵添加一个偏置向量，该偏置向量的维度与矩阵的列数相同。偏置向量可以通过训练获得，也可以预先设定。在计算注意力权重时，添加偏置向量可以使模型更好地捕捉输入序列之间的关系，从而提高模型的性能。

<p align="center">
<img src="./imgs/MultiheadAttention_add_bias_kv.svg"
    width="2000" /></p>


# 图解add_zero_attn
在自注意力机制中，我们会将输入序列中的每个元素都视为一个向量，并将它们传递给多个注意力头进行计算。每个注意力头都会将输入序列中的每个位置都与所有其他位置进行比较，并计算它们之间的相似度得分，从而为每个位置分配一个注意力权重。

在计算相似度得分时，通常会使用点积注意力或其他形式的注意力计算方法。这些计算方法涉及向输入序列中的每个位置分配一个向量，分别用作查询向量、键向量和值向量。查询向量用于确定当前位置所关注的内容，而键向量和值向量则用于计算该位置与其他位置之间的相似度得分。

在add_zero_attn技术中，我们会向输入序列中添加一个额外的“零位置”，它的向量值为零。这个零位置会在计算注意力权重时被考虑，从而确保输入序列中的所有位置都有相应的注意力权重。

这个零位置的目的是为了解决输入序列中可能存在一些不重要或无关紧要的位置的问题。如果输入序列中有些位置与当前任务或应用无关，那么它们的注意力权重可能会分散在其他位置上，从而导致模型的性能下降。通过向输入序列中添加一个零位置，我们可以确保所有位置都有相应的注意力权重，从而提高模型的准确性和鲁棒性。

需要注意的是，添加零位置可能会使输入序列的长度增加一，从而增加计算量。为了减少计算量，通常会将零位置的向量值设置为常数，而不是通过训练来确定其值。这样做可以使计算零位置的注意力权重变得更加高效。

总之，add_zero_attn技术可以帮助深度学习模型在自注意力机制中更好地处理输入序列中的位置关系，提高模型的性能和准确性。

简言之：  
输入序列的某些位置对于任务是没作用的，通过那我通过什么样的方式把这个位置的value降权呢？就是靠add_zero_attn，让这个位置的q查询其他位置的k时比较小甚至是负数，那么attn到zero位置的k时（k是0向量），权重就相对大了（虽然权重恒为0，但别的位置还可能是负数），再经过Softmax后得到归一化的权重，然后带权相加，而zero位置的v也是0向量，带权相加就相当于对该位置的value做了降权。

<p align="center">
<img src="./imgs/MultiheadAttention_add_zero_attn.svg"
    width="2000" /></p>
